# Assignment 1

**Due to**: 11/01/2022 (dd/mm/yyyy)

If you deliver it by 11/12/2021 your assignment will be graded by 11/01/2022.


**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Summary**: Part-of Speech (POS) tagging as Sequence Labelling using Recurrent Neural Architectures

# Execution
https://www.kaggle.com/code/tanyadayanand/pos-tagging-using-rnn

A bunch of libraries and functions that will be used throughout the notebook.


In [1]:
import re
import pandas as pd
import numpy as np
import os
import urllib.request
import zipfile
import progressbar

import nltk
import sklearn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, GRU
from sklearn.metrics import classification_report

nltk.download('treebank')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
def pre_process(df,string):
  def text_pre_process(text):
      ret = re.sub("RT @(.)+?:\s|(&#[0-9]+;)|@([\w\-]+)|(#)\S+|(http)s?\S+|&gt;|^\s+|\b\s+|\n", "", text)
      ret = re.sub("\s\s+|[^a-zA-Z\d\s:]" , " ", ret).rstrip().lower()
      return ret
  return df[string].apply(text_pre_process)

def Encoding(df,Tags):
  label_encoder = sklearn.preprocessing.LabelEncoder()
  X_lab = label_encoder.fit_transform(df)
  OneHot_encoder = sklearn.preprocessing.OneHotEncoder()
  X = OneHot_encoder.fit_transform(X_lab.reshape(-1,1)).toarray()
  dfOneHot = pd.DataFrame(X, columns = [i for i in Tags])
  df.reset_index(inplace=True,drop=True)
  df = pd.concat([df,dfOneHot],axis=1)
  return df

def create_embedding_matrix(filepath, word_index, embedding_dim):
  vocab_size = len(word_index)+1
  embedding_matrix = np.zeros((vocab_size,embedding_dim))

  with open(filepath, encoding='utf-8') as f:
    for line in f:
      word, *vector = line.split()
      if word in word_index:
        idx = word_index[word]
        embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]
  return embedding_matrix

## 1. Corpus
### 1.1 Pre-processing

From the original tags list we removed all the symbols and english punctuation plus:
- FW, Foreign Word, because there are no examples in the test set;
- UH, Interjection, because there are no examples in the test set;
- LS, List Item Marker, because there are no examples in the test set (and because it denotes symbols as well);

In [35]:
# Get the files' list
fileids = nltk.corpus.treebank.fileids()

# Get the Penn Treebank corpus and tokenize the text
train_corpus = nltk.corpus.treebank.tagged_sents(fileids[:100])
val_corpus = nltk.corpus.treebank.tagged_sents(fileids[100:150])
test_corpus = nltk.corpus.treebank.tagged_sents(fileids[150:])

# Flatten the lists
train_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(train_corpus) for item in sublist if item[1] != '-NONE-']
val_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(val_corpus) for item in sublist if item[1] != '-NONE-']
test_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(test_corpus) for item in sublist if item[1] != '-NONE-']

In [36]:
train_df = pd.DataFrame(train_corpus, columns = ['word', 'tag', 'sentence'])

train_df['word'] = pre_process(train_df,'word')

remove = [':', '#', '"', '$', '-LRB-', '-RRB-', ',', '.', "''", '``', 'SYM']
for r in remove:
  train_df['tag'].replace(r, np.nan, inplace=True)
train_df.dropna(inplace=True)

tags_train = sorted(list(set([x for x in train_df.tag])))

print(train_df.shape)
train_df.describe()

(41274, 3)


,word,tag,sentence
count,41274,41274,41274
unique,7359,35,1963
top,the,NN,1854
freq,2329,6270,171


In [37]:
val_df = pd.DataFrame(val_corpus, columns = ['word', 'tag', 'sentence'])

val_df['word'] = pre_process(val_df,'word')

remove = [':', '#', '"', '$', '-LRB-', '-RRB-', ',', '.', "''", '``', 'SYM']
for r in remove:
  val_df['tag'].replace(r, np.nan, inplace=True)
val_df.dropna(inplace=True)


tags_val = sorted(list(set([x for x in val_df.tag])))

print(val_df.shape)
val_df.describe()

(27418, 3)


,word,tag,sentence
count,27418,27418,27418
unique,5385,35,1299
top,the,NN,339
freq,1670,4513,75


In [38]:
test_df = pd.DataFrame(test_corpus, columns = ['word', 'tag', 'sentence'])

test_df['word'] = pre_process(test_df,'word')

remove = [':', '#', '"', '$', '-LRB-', '-RRB-', ',', '.', "''", '``', 'SYM']
for r in remove:
  test_df['tag'].replace(r, np.nan, inplace=True)
test_df.dropna(inplace=True)


tags_test = sorted(list(set([x for x in test_df.tag])))

print(test_df.shape)
test_df.describe()

(13676, 3)


,word,tag,sentence
count,13676,13676,13676
unique,3383,32,652
top,the,NN,231
freq,765,2383,51


In [39]:
print('Train:',len(tags_train))
print('Val:',len(tags_val))
print('Test:',len(tags_test))

if len(tags_test) != len(tags_val) or len(tags_test) != len(tags_train):
  print('Mismatching number of classes.')
else:
  print('\nTags:')
  for tag in tags_train:
    print(f'-{tag}')

Train: 35
Val: 35
Test: 32
Mismatching number of classes.


In [40]:
missing_classes_train = [x for x in tags_train if x not in tags_test]
missing_classes_val   = [x for x in tags_val if x not in tags_test]

missing_classes = list(set(missing_classes_train + missing_classes_val))

for cl in missing_classes:
  train_df = test_df[test_df.tag != cl]
  val_df = test_df[test_df.tag != cl]

tags_train = sorted(list(set([x for x in train_df.tag])))
tags_val = sorted(list(set([x for x in val_df.tag])))
tags_test = sorted(list(set([x for x in test_df.tag])))

print('Train:',len(tags_train))
print('Val:',len(tags_val))
print('Test:',len(tags_test))

if len(tags_test) != len(tags_val) or len(tags_test) != len(tags_train):
  print('Mismatching number of classes.')
else:
  print('\nTags:')
  for tag in tags_train:
    print(f'-{tag}')

Train: 32
Val: 32
Test: 32

Tags:
-CC
-CD
-DT
-EX
-IN
-JJ
-JJR
-JJS
-MD
-NN
-NNP
-NNPS
-NNS
-PDT
-POS
-PRP
-PRP$
-RB
-RBR
-RBS
-RP
-TO
-VB
-VBD
-VBG
-VBN
-VBP
-VBZ
-WDT
-WP
-WP$
-WRB


In [41]:
label_encoder = LabelEncoder()

train_df['tag'] = label_encoder.fit_transform(train_df['tag'])
test_df['tag']  = label_encoder.fit_transform(test_df['tag'])
val_df['tag']   = label_encoder.fit_transform(val_df['tag'])

print(train_df['tag'].unique())
print(val_df['tag'].unique())
print(test_df['tag'].unique())

[10 23 15  1 12  0  4  9 16  5  2 27 14 21  8 22 24 17 25 29 26 31 28  6
  7 20 11 18  3 19 30 13]
[10 23 15  1 12  0  4  9 16  5  2 27 14 21  8 22 24 17 25 29 26 31 28  6
  7 20 11 18  3 19 30 13]
[10 23 15  1 12  0  4  9 16  5  2 27 14 21  8 22 24 17 25 29 26 31 28  6
  7 20 11 18  3 19 30 13]


In [44]:
X_train = train_df.word
y_train = train_df.tag
y_train = Encoding(y_train, tags_train)

X_val = val_df.word
y_val = val_df.tag
y_val = Encoding(y_val, tags_val)

X_test = test_df.word
y_test = test_df.tag
y_test = Encoding(y_test, tags_test)

y_train.drop('tag',inplace=True,axis=1)
y_test.drop('tag',inplace=True,axis=1)
y_val.drop('tag',inplace=True,axis=1)

print(y_train.shape)
print(y_train.head())

print(y_val.shape)
print(y_val.head())

print(y_test.shape)
print(y_test.head())

(13676, 32)
    CC   CD   DT   EX   IN   JJ  JJR  JJS   MD   NN  ...   VB  VBD  VBG  VBN  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

   VBP  VBZ  WDT   WP  WP$  WRB  
0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 32 columns]
(13676, 32)
    CC   CD   DT   EX   IN   JJ  JJR  JJS   MD   NN  ...   VB  VBD  VBG  VBN  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  

In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

num_words = 9000
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= num_words}
tokenizer.word_index[tokenizer.oov_token] = num_words + 1

X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1

In [46]:
max_len = 100
X_train = pad_sequences(X_train,padding='post',maxlen=max_len)
X_val = pad_sequences(X_val,padding='post',maxlen=max_len)
X_test = pad_sequences(X_test,padding='post',maxlen=max_len)

## 2. GloVe 
GloVe (Global Vectors for Word Representation) is a method for learning vector representations of words, called "word embeddings," from a large corpus of text. Word embeddings are numerical representations of words that capture the semantic relationships between words in a continuous, low-dimensional space. They are commonly used as input to natural language processing models, such as language translation and language modeling.

GloVe works by learning the co-occurrence statistics of words in a corpus, and using this information to learn word embeddings that capture the semantic relationships between words. The GloVe method produces word embeddings that are trained on a global corpus, as opposed to embeddings that are trained on a specific task or dataset.

There are different versions of the GloVe word embeddings, including 50-dimensional, 100-dimensional, and 200-dimensional embeddings. The 50-dimensional version of GloVe embeddings may be better in some applications because they have a lower dimensionality, which can make them easier to work with and more computationally efficient.

In [47]:
pbar = None
def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

# Download the GloVe embeddings file
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
urllib.request.urlretrieve(url, 'glove.6B.zip', show_progress)

# Extract the zip file
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall()
zip_ref.close()

100% (862182613 of 862182613) |##########| Elapsed Time: 0:02:39 Time:  0:02:39


In [48]:
# Load the GloVe embeddings into a dictionary
embedding_dict = {}
with open('glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_dict[word] = coefs

# Print the number of words in the embeddings dictionary
print(f'Found {len(embedding_dict)} word vectors.')

Found 400000 word vectors.


In [50]:
def find_closest_embeddings(embedding):
    return sorted(embedding_dict.keys(), key=lambda word: np.linalg.norm(embedding_dict[word]- embedding))[:5]

find_closest_embeddings(embedding_dict['iphone'])

['iphone', 'ipad', 'smartphone', 'ipod', 'android']

In [51]:
input_dim = X_train.shape[1]
embedding_dim = 50
embedding_matrix = create_embedding_matrix(f'glove.6B.{embedding_dim}d.txt', tokenizer.word_index, embedding_dim)

## 3. Model
### 3.1 Baseline (MACRO f1 0.84)
Bidirectional LSTM layers are able to process sequential data in both the forward and backward directions, which can allow the model to capture contextual information from both the past and the future. This can be particularly useful for natural language processing tasks, where the meaning of a word can depend on the context in which it is used.

In [52]:
# Define the model
model = tf.keras.Sequential(name='Baseline')

# Add the Embedding layer
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, \
                    weights = [embedding_matrix], input_length = max_len, trainable=True))

# Add the Bidirectional LSTM layer
model.add(Bidirectional(LSTM(units=128)))

# Add the Dense/Fully-Connected layer
model.add(Dense(units=len(tags_train), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

Model: "Baseline"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           158450    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              183296    
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                8224      
                                                                 
Total params: 349,970
Trainable params: 349,970
Non-trainable params: 0
_________________________________________________________________


In [ ]:
results = model.fit(X_train, y_train, epochs=10, verbose = True, validation_data=(X_val,y_val), batch_size=32)

Epoch 1/10
421/428 [============================>.] - ETA: 1s - loss: 1.5198 - accuracy: 0.5788

In [ ]:
y_pred = model.predict(X_test)

428/428 [==============================] - 14s 32ms/step


In [ ]:
th = 0.1
y_pred[y_pred >= th] = 1 
y_pred[y_pred  < th] = 0

print(classification_report(y_test, y_pred, target_names = tags_train, zero_division=True))

              precision    recall  f1-score   support

          CC       0.99      0.96      0.98       366
          CD       1.00      1.00      1.00       858
          DT       0.91      1.00      0.95      1335
          EX       0.83      1.00      0.91         5
          IN       0.92      1.00      0.96      1630
          JJ       0.85      0.99      0.91       918
         JJR       0.77      1.00      0.87        59
         JJS       0.89      1.00      0.94        31
          MD       0.97      1.00      0.98       167
          NN       0.86      1.00      0.92      2383
         NNP       0.77      0.99      0.87      1504
        NNPS       0.35      1.00      0.51        44
         NNS       0.97      0.99      0.98       941
         PDT       0.22      0.50      0.31         4
         POS       0.94      0.95      0.94       152
         PRP       0.99      1.00      0.99       192
        PRP$       1.00      1.00      1.00        99
          RB       0.84    

### 3.2 GRU 
Unica che non funzia dio po

In [ ]:
from tensorflow.keras.layers import TimeDistributed

# Define the model
model = tf.keras.Sequential(name='GRU')

# Add the Embedding layer
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, \
                    weights = [embedding_matrix], input_length = max_len, trainable=True))

# Add the GRU layer
model.add(GRU(units=32, return_sequences=True))

# Add the Dense/Fully-Connected layer
# model.add(Dense(units=len(tags_train), activation='softmax'))
model.add(TimeDistributed(Dense(len(tags_train), activation='softmax')))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 50, 50)            158450    
                                                                 
 gru_8 (GRU)                 (None, 50, 32)            8064      
                                                                 
 time_distributed_3 (TimeDis  (None, 50, 32)           1056      
 tributed)                                                       
                                                                 
Total params: 167,570
Trainable params: 167,570
Non-trainable params: 0
_________________________________________________________________


In [ ]:
results = model.fit(X_train, y_train, epochs=10, verbose = True, validation_data=(X_val,y_val), batch_size=32)

Epoch 1/10


ValueError: ignored

In [ ]:
y_pred = model.predict(X_test)

428/428 [==============================] - 6s 13ms/step


In [ ]:
th = 0.1
y_pred[y_pred >= th] = 1 
y_pred[y_pred  < th] = 0

print(classification_report(y_test, y_pred, target_names = tags_train, zero_division=True))

              precision    recall  f1-score   support

          CC       1.00      0.00      0.00       366
          CD       1.00      0.00      0.00       858
          DT       0.10      1.00      0.18      1335
          EX       1.00      0.00      0.00         5
          IN       0.12      1.00      0.21      1630
          JJ       1.00      0.00      0.00       918
         JJR       1.00      0.00      0.00        59
         JJS       1.00      0.00      0.00        31
          MD       1.00      0.00      0.00       167
          NN       0.17      1.00      0.30      2383
         NNP       0.11      1.00      0.20      1504
        NNPS       1.00      0.00      0.00        44
         NNS       1.00      0.00      0.00       941
         PDT       1.00      0.00      0.00         4
         POS       1.00      0.00      0.00       152
         PRP       1.00      0.00      0.00       192
        PRP$       1.00      0.00      0.00        99
          RB       1.00    

### 3.3 Additional LSTM layer (MACRO f1 0.82) 
Using two bidirectional LSTM layers can allow the model to learn more complex patterns in the data and make more accurate predictions. 
However, they can increase the computational complexity of our model, which may require more computational resources to train.

Indeed, here the train was slower and the results similar to the baseline architecture.

In [ ]:
# Define the model
model = tf.keras.Sequential(name='Additional_LSTM')

# Add the Embedding layer
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, \
                    weights = [embedding_matrix], input_length = max_len, trainable=True))

# Add the Bidirectional LSTM layer
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))

# Add another LSTM layer
model.add(Bidirectional(LSTM(units=64)))

# Add the Dense/Fully-Connected layer
model.add(Dense(units=len(tags_train), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

Model: "Additional_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            158450    
                                                                 
 bidirectional_1 (Bidirectio  (None, 50, 128)          58880     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 32)                4128      
                                                                 
Total params: 320,274
Trainable params: 320,274
Non-trainable params: 0
_________________________________________________________________


In [ ]:
results = model.fit(X_train, y_train, epochs=10, verbose = True, validation_data=(X_val,y_val), batch_size=32)

Epoch 1/10
428/428 [==============================] - 68s 142ms/step - loss: 1.5208 - accuracy: 0.5648 - val_loss: 0.8600 - val_accuracy: 0.7560
Epoch 2/10
428/428 [==============================] - 53s 124ms/step - loss: 0.7112 - accuracy: 0.7964 - val_loss: 0.5132 - val_accuracy: 0.8484
Epoch 3/10
428/428 [==============================] - 56s 131ms/step - loss: 0.4941 - accuracy: 0.8539 - val_loss: 0.3821 - val_accuracy: 0.8882
Epoch 4/10
428/428 [==============================] - 50s 118ms/step - loss: 0.3723 - accuracy: 0.8909 - val_loss: 0.2882 - val_accuracy: 0.9165
Epoch 5/10
428/428 [==============================] - 50s 118ms/step - loss: 0.3013 - accuracy: 0.9108 - val_loss: 0.2428 - val_accuracy: 0.9283
Epoch 6/10
428/428 [==============================] - 52s 122ms/step - loss: 0.2544 - accuracy: 0.9221 - val_loss: 0.2105 - val_accuracy: 0.9355
Epoch 7/10
428/428 [==============================] - 62s 145ms/step - loss: 0.2238 - accuracy: 0.9291 - val_loss: 0.1886 - val_ac

In [ ]:
y_pred = model.predict(X_test)

428/428 [==============================] - 12s 25ms/step


In [ ]:
th = 0.1
y_pred[y_pred >= th] = 1 
y_pred[y_pred  < th] = 0

print(classification_report(y_test, y_pred, target_names = tags_train, zero_division=True))

              precision    recall  f1-score   support

          CC       0.99      0.96      0.98       366
          CD       0.98      1.00      0.99       858
          DT       0.91      1.00      0.95      1335
          EX       0.71      1.00      0.83         5
          IN       0.76      1.00      0.86      1630
          JJ       0.85      0.99      0.91       918
         JJR       0.77      1.00      0.87        59
         JJS       0.84      1.00      0.91        31
          MD       0.97      1.00      0.98       167
          NN       0.86      0.99      0.92      2383
         NNP       0.81      0.97      0.88      1504
        NNPS       0.71      0.89      0.79        44
         NNS       0.95      0.99      0.97       941
         PDT       0.22      0.50      0.31         4
         POS       0.93      0.95      0.94       152
         PRP       0.99      1.00      0.99       192
        PRP$       0.99      1.00      0.99        99
          RB       0.86    

### 3.4 Additional dense layer (MACRO f1 0.85)

Using two dense layers, one with a non-linear activation function and one with a softmax activation function, is a common pattern in neural network architectures for classification tasks.

The purpose of the non-linear dense layer is to introduce non-linearity into the model, which can allow the model to learn more complex patterns in the data. Common choices for the activation function in this layer include ReLU (Rectified Linear Unit), sigmoid, and tanh.

The purpose of the softmax dense layer is to produce a probability distribution over the possible classes. The softmax activation function transforms the output of the preceding layer into a probability distribution, where the sum of the probabilities is equal to 1. This is useful for classification tasks, where you want to predict the probability that an input belongs to each of the possible classes. Using two dense layers in this way can allow the model to learn more complex patterns in the data and make more accurate predictions.

In [ ]:
# Define the model
model = tf.keras.Sequential(name='Baseline')

# Add the Embedding layer
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, \
                    weights = [embedding_matrix], input_length = max_len, trainable=True))

# Add the Bidirectional LSTM layer
model.add(Bidirectional(LSTM(units=128)))

# Add another Dense layer
model.add(Dense(units=256, activation='relu'))

# Add the Dense/Fully-Connected layer
model.add(Dense(units=len(tags_train), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

Model: "Baseline"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 50)            158450    
                                                                 
 bidirectional_4 (Bidirectio  (None, 256)              183296    
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
 dense_6 (Dense)             (None, 32)                8224      
                                                                 
Total params: 415,762
Trainable params: 415,762
Non-trainable params: 0
_________________________________________________________________


In [ ]:
results = model.fit(X_train, y_train, epochs=10, verbose = True, validation_data=(X_val,y_val), batch_size=32)

Epoch 1/10
428/428 [==============================] - 63s 138ms/step - loss: 1.2939 - accuracy: 0.6378 - val_loss: 0.6953 - val_accuracy: 0.8005
Epoch 2/10
428/428 [==============================] - 58s 136ms/step - loss: 0.5945 - accuracy: 0.8265 - val_loss: 0.4121 - val_accuracy: 0.8823
Epoch 3/10
428/428 [==============================] - 58s 136ms/step - loss: 0.4019 - accuracy: 0.8820 - val_loss: 0.3018 - val_accuracy: 0.9100
Epoch 4/10
428/428 [==============================] - 60s 139ms/step - loss: 0.3035 - accuracy: 0.9081 - val_loss: 0.2439 - val_accuracy: 0.9258
Epoch 5/10
428/428 [==============================] - 59s 139ms/step - loss: 0.2442 - accuracy: 0.9236 - val_loss: 0.1910 - val_accuracy: 0.9387
Epoch 6/10
428/428 [==============================] - 67s 157ms/step - loss: 0.2069 - accuracy: 0.9295 - val_loss: 0.1682 - val_accuracy: 0.9443
Epoch 7/10
428/428 [==============================] - 66s 153ms/step - loss: 0.1818 - accuracy: 0.9359 - val_loss: 0.1488 - val_ac

In [ ]:
y_pred = model.predict(X_test)

428/428 [==============================] - 16s 35ms/step


In [ ]:
th = 0.1
y_pred[y_pred >= th] = 1 
y_pred[y_pred  < th] = 0

print(classification_report(y_test, y_pred, target_names = tags_train, zero_division=True))

              precision    recall  f1-score   support

          CC       0.81      1.00      0.89       366
          CD       1.00      1.00      1.00       858
          DT       0.99      0.99      0.99      1335
          EX       0.83      1.00      0.91         5
          IN       0.95      0.99      0.97      1630
          JJ       0.84      0.99      0.91       918
         JJR       0.79      1.00      0.88        59
         JJS       0.91      1.00      0.95        31
          MD       0.97      1.00      0.98       167
          NN       0.88      0.99      0.93      2383
         NNP       0.84      0.97      0.90      1504
        NNPS       0.59      0.95      0.73        44
         NNS       0.96      1.00      0.98       941
         PDT       0.12      0.50      0.20         4
         POS       0.94      0.95      0.94       152
         PRP       1.00      1.00      1.00       192
        PRP$       1.00      1.00      1.00        99
          RB       0.75    